# Measuring Quantum Hamiltonians with Shallow Circuits using Decision Diagrams


In [1]:
import sys
print(sys.version)
import decimal as d
from typing import Dict, List, Callable, Hashable, Tuple, Set, Optional
try:
    from math import prod
except ImportError:
    from functools import reduce 
    import operator
    def prod(iterable):
        return reduce(operator.mul, iterable, 1)

import networkx as nx

PauliTerm = str
PauliOp = str
Hamiltonian = Dict[PauliTerm, d.Decimal]
DDSampler = Callable[[nx.MultiDiGraph, Hashable], Tuple[d.Decimal, PauliTerm]]
Node = Hashable

3.8.5 (default, Jan 27 2021, 15:41:15) 
[GCC 9.3.0]


In [2]:
def get_included_terms(ppaths: List[PauliTerm], meas: PauliTerm) -> List[PauliTerm]:
    """return set of keys included by measurement basis"""
    answer = []
    for p in ppaths:
        if len(p) != len(meas):
            continue
        else:
            is_included = True
            for i in range(len(p)):
                if p[i] != "I" and p[i] != meas[i]:
                    is_included = False
                    break
            if is_included:
                answer.append(p)
    return answer

# One-Shot Variance

$P = \{I, X, Y, Z \}$

$\mathcal{P} = \hat{P} = \{X, Y, Z \}$ 

- $\mathcal{P}$ is hard to distinctively write by hand, so if my notes have $\hat{P}$ somewhere it's the same as $\mathcal{P}$

$\operatorname{Var}(\nu)
    =
    \underbrace{\left(
    \sum_{P,Q} \alpha_P \alpha_Q 
        g(P,Q,\beta)
            \operatorname{Tr}(PQ\rho)
    \right)}_{\text{first part}}
    -
    {\underbrace{\left(
    \operatorname{Tr}(H\rho)
    \right)}_\text{second part}}^2$
    
- Better names for *first_part* and *second_part*?
    
$\operatorname{Tr}(PQ\rho) = \langle\psi_\rho| \operatorname{multiply\_terms}(P,Q) |\psi_\rho\rangle$ 

$g(P, Q, \beta) 
   = \frac{1}{\zeta(P, \beta)} \frac{1}{\zeta(Q, \beta)} \sum_{B \in \operatorname{Lift}(P) \cap \operatorname{Lift}(Q)}\beta(B) 
   = \frac{1}{\zeta(P, \beta)} \frac{1}{\zeta(Q, \beta)} \zeta(\operatorname{merge\_terms}(P,Q), \beta)$
   
- What should happen if one of the $\zeta$s in the divisor is 0?

$\zeta(P, \beta) = \sum_{B \in \operatorname{Lift}(P)} \beta(B)$

- The $\zeta$ function is the same as `dd.compute_covered_prob(graph, 0, P)` for coefficient-DD (0 means starting at the top node)
- The $\zeta$ function is the same as `dd.count_covering_paths(graph, 0, P)) / total_paths` for uniform-path-DD (0 means starting at the top node)

$\operatorname{Lift}(P) = \{ B \in \mathcal{P}^n \mid B_i = P_i \text{ for every } i \in \operatorname{supp}(P) \}$

$\operatorname{supp}(Q) = \{ i \mid Q_i \neq I \}$

In [3]:
import itertools
import time
import pathlib
import copy

import networkx as nx
import numpy as np
import qutip

import src.sampling as dd


def read_hamiltonian(filename: str) -> Hamiltonian:
    H = {}
    lineno = 0
    with open(filename, "r") as fp:
        for line in fp:
            if lineno % 2 == 0:
                key = line.rstrip()
            else:
                val = d.Decimal(line.rstrip())
                H[key] = val
            lineno += 1
    return H


def convert_H_to_matrix(H: Hamiltonian):
    """converting dictionary to matrix, and returning the j-th eigenpairs"""
    paulis = {"I": qutip.identity(2),
              "X": qutip.sigmax(),
              "Y": qutip.sigmay(),
              "Z": qutip.sigmaz()}
    for k in H.keys():
        nQubits = len(k)
        break
        
    matrix = None
    for pauli_term, coefficient in H.items():
        if matrix is None:
            matrix = float(coefficient) * qutip.tensor( [ paulis[p] for p in pauli_term ] )
        else:
            matrix += float(coefficient) * qutip.tensor( [ paulis[p] for p in pauli_term ] )
            
    eigenpair = matrix.eigenstates()
    return matrix, (eigenpair[0][0], eigenpair[1][0])


def pauli_kron_state_product(pauli_string: PauliTerm, vec1: np.ndarray) -> np.ndarray:
    """
        THIS MUST BE USED FOR KRONECKER PRODUCT OF PAULI MATRICES
        Using generalized vec-trick optimized for Pauli matrices for kron-vec multiplication
    """
    n = len(pauli_string)
    # N = 2**n
    # halfN = N // 2
    vec = copy.deepcopy(vec1)
    vec = vec.reshape((2, -1), order="F")  # reshape
    for i, p in enumerate(pauli_string[::-1]):
        # apply Pi
        if p == "Z":
            # multiply the second row by -1
            vec[1, :] *= -1
        elif p == "X" or p == "Y":
            # swap the rows
            vec[[0, 1]] = vec[[1, 0]]
            if p == "Y":
                vec *= 1.0j
                vec[0, :] *= -1
        if i < n - 1:
            vec = vec.reshape((-1, 2), order="C").T

    return np.ravel(vec)


def zeta(graph: nx.MultiDiGraph, B: PauliTerm) -> d.Decimal:
    prob = dd.compute_covered_prob(graph, 0, B)
    return prob


def merge_terms(P: PauliTerm, Q: PauliTerm) -> PauliTerm:
    """
        Combine terms such that Lift(combine(P, Q)) = Lift(P)∩Lift(Q)
    """
    assert len(P) == len(Q)
    res = []
    for p, q in zip(P, Q):
        if p == q:
            res.append(p)
        elif p == 'I':
            res.append(q)
        elif q == 'I':
            res.append(p)
        else:
            raise RuntimeError('terms not compatible')
    return ''.join(res)

def multiply_terms(P: PauliTerm, Q: PauliTerm) -> PauliTerm:
    """X X = Y Y = Z Z = I and X I = X I = X and Y I = I Y = Y and Z I = I Z = Z. """
    assert len(P) == len(Q)
    res = []
    for p, q in zip(P, Q):
        if p == q:
            res.append('I')
        elif p == 'I':
            res.append(q)
        elif q == 'I':
            res.append(p)
        else:
            raise RuntimeError('terms not compatible')
    return ''.join(res)


def g(graph: nx.MultiDiGraph, P: PauliTerm, Q: PauliTerm) -> d.Decimal:
    zeta_P = zeta(graph, P)
    zeta_Q = zeta(graph, Q)
    zeta_PcapQ = zeta(graph, combine(P, Q))
       
    assert zeta_P != 0 and zeta_Q != 0, f'{P} {zeta_P} | {Q} {zeta_Q} | {combine(P,Q)} {zeta_PcapQ}'
    
    return (d.Decimal(1) / zeta_P) * (d.Decimal(1) / zeta_Q) * zeta_PcapQ

    
def one_shot_variance_coeff(H: Hamiltonian, Hmatrix: np.ndarray, graph: nx.MultiDiGraph, input_state: np.ndarray, exact_value: d.Decimal=None) -> d.Decimal:
    """beta is according to the coefficients"""
    paulis = {"I": qutip.identity(2),
          "X": qutip.sigmax(),
          "Y": qutip.sigmay(),
          "Z": qutip.sigmaz()}
    
    # the input_state is a pure state, i.e. a vector
    rho = input_state.full() @ input_state.conj().trans().full()
    
    first_part = d.Decimal(0)
    zeta_cache = {}
    psi_bra = input_state.conj().trans().full()
    psi_ket = input_state.full()
    
    # if P or Q is not in H (having an implicit coefficent of 0) we don't need to consider them
    for P, Q in itertools.product(H, repeat=2):
        # if P and Q are not compatible we can skip them as well
        try:
            merge_terms(P, Q)
        except RuntimeError:
            continue
            
        #print(f'Checking {P} and {Q}')
        coeffs = H[P] * H[Q] # 𝛼_𝑃 * 𝛼_𝑄
        
        if coeffs == 0:
            continue
        
        #g_PQ = g(graph, P, Q)
        
        if P not in zeta_cache:
            zeta_cache[P] = zeta(graph, P)
        zeta_P = zeta_cache[P]
        if Q not in zeta_cache:
            zeta_cache[Q] = zeta(graph, Q)
        zeta_Q = zeta_cache[Q]
        
        if merge_terms(P, Q) not in zeta_cache:
            zeta_cache[merge_terms(P, Q)] = zeta(graph, merge_terms(P, Q))
        zeta_PcapQ = zeta_cache[merge_terms(P, Q)]
        
        #zeta_P = zeta(graph, P)
        #zeta_Q = zeta(graph, Q)
        #zeta_PcapQ = zeta(graph, merge_terms(P, Q))
                
        if zeta_P == 0 or zeta_Q == 0 or zeta_PcapQ == 0:
            continue

        #g_PQ = (d.Decimal(1) / zeta_P) * (d.Decimal(1) / zeta_Q) * zeta_PcapQ
        g_PQ = zeta_PcapQ / min(zeta_P, zeta_Q) / max(zeta_P, zeta_Q)
        
        #P_matrix = qutip.tensor( [ paulis[p] for p in P ] ).full()
        #Q_matrix = qutip.tensor( [ paulis[p] for p in Q ] ).full()
        #trace = np.trace(P_matrix @ Q_matrix @ rho)
        
        
        p_alpha_psi_ket = np.atleast_2d(pauli_kron_state_product(multiply_terms(P, Q), psi_ket)).T
        trace = (psi_bra @ p_alpha_psi_ket)[0][0]
        
        #print(trace)
        
        first_part += coeffs * g_PQ * d.Decimal(trace.real)

    if exact_value:
        second_part = exact_value
    else:
        second_part = np.trace(np.matmul(Hmatrix.full(), rho))
        assert second_part.imag <= 0.001, f'second part == {second_part}'
        second_part = d.Decimal(second_part.real)

    return first_part, second_part, first_part - second_part**2


def one_shot_variance_paths(H: Hamiltonian, Hmatrix: np.ndarray, graph: nx.MultiDiGraph, input_state: np.ndarray, exact_value: d.Decimal=None) -> d.Decimal:
    """beta is according to the coefficients"""
    paulis = {"I": qutip.identity(2),
          "X": qutip.sigmax(),
          "Y": qutip.sigmay(),
          "Z": qutip.sigmaz()}
    
    # the input_state is a pure state, i.e. a vector
    rho = input_state.full() @ input_state.conj().trans().full()
    total_paths = len(dd.dfs(graph, 0, '', set()))
    
    first_part = d.Decimal(0)
    zeta_cache = {}
    psi_bra = input_state.conj().trans().full()
    psi_ket = input_state.full()
    
    # if P or Q is not in H (having an implicit coefficent of 0) we don't need to consider them
    for P, Q in itertools.product(H, repeat=2):
        # if P and Q are not compatible we can skip them as well
        try:
            merge_terms(P, Q)
        except RuntimeError:
            continue
            
        #print(f'Checking {P} and {Q}')
        coeffs = H[P] * H[Q] # 𝛼_𝑃 * 𝛼_𝑄
        
        if coeffs == 0:
            continue
        
        #g_PQ = g(graph, P, Q)
        
        if P not in zeta_cache:
            zeta_cache[P] = d.Decimal(dd.count_covering_paths(graph, 0, P)) / total_paths
        zeta_P = zeta_cache[P]
        if Q not in zeta_cache:
            zeta_cache[Q] = d.Decimal(dd.count_covering_paths(graph, 0, Q)) / total_paths
        zeta_Q = zeta_cache[Q]
        if merge_terms(P, Q) not in zeta_cache:
            zeta_cache[merge_terms(P, Q)] = d.Decimal(dd.count_covering_paths(graph, 0, merge_terms(P, Q))) / total_paths
        zeta_PcapQ = zeta_cache[merge_terms(P, Q)]
        
        # It can happen, that a PauliTerm from Hdict is not in the DD
        # A few lines below this would lead to a division by 0 error
        if zeta_P == 0 or zeta_Q == 0:
            continue

        g_PQ = (d.Decimal(1) / zeta_P) * (d.Decimal(1) / zeta_Q) * zeta_PcapQ
        
        #P_matrix = qutip.tensor( [ paulis[p] for p in P ] ).full()
        #Q_matrix = qutip.tensor( [ paulis[p] for p in Q ] ).full()
        #trace = np.trace(P_matrix @ Q_matrix @ rho)
        
        p_alpha_psi_ket = np.atleast_2d(pauli_kron_state_product(multiply_terms(P,Q), psi_ket)).T
        trace = (psi_bra @ p_alpha_psi_ket)[0][0]
        
        #print(trace)
        
        first_part += coeffs * g_PQ * d.Decimal(trace.real)
    
    if exact_value:
        second_part = exact_value
    else:
        second_part = np.trace(np.matmul(Hmatrix.full(), rho))
        assert second_part.imag <= 0.001, f'second part == {second_part}'
        second_part = d.Decimal(second_part.real)

    return first_part, second_part, first_part - second_part**2

# Algorithm to compute optimal $\mathbf{p}$ minimizing variances

Input: Hambiltonian $H$, Decision Diagram $g$, small $\delta > 0$ (e.g. 0.005). Basically, $\mathbf{p}$ is the edge weights in the graph.

- $v$ is a vertex and $v_i$ with $i \in \{X, Y, Z\}$ is an out-edge from vertex $v$
- $\Pr_g[v_i]$ is the edge weight of edge $v_i$ in graph $g$. Given $v$: $\sum_{i \in \operatorname{out}(v)}\Pr_g[v_i] = 1$  
  $\Pr_g(b)$ is the probability of path $b \in \{X, Y, Z\}^n$ in graph g  
  graph $g'$ is a copy of $g$ where the weights are adjusted (think $\mathbf{p}'$)
- $\zeta_g(Q) = \sum_{b \in \operatorname{Lift}(Q)}{\Pr}_g(b)$
- $\zeta_g(Q, v_i) = \sum_{b \in \operatorname{Lift}(Q) \text{ and path contains } v_i}{\Pr}_g(b)$
- $T_{g}(H, v_i) = \sum_{Q \in H} (\alpha_Q)^2 \frac{\zeta_g(Q, v_i)}{(\zeta_g(Q))^2}$

Initialize edge weights in $g$, using heuristic or just random. 

Repeat the following steps until $g$ converges  
- for each vertex $v$ in $g$
  - calculate normalizer:  
    $$\lambda_v = \sum_{v_i \in \operatorname{out}(v)} \sum_{Q\in H} (\alpha_Q)^2 \frac{\zeta_g(Q, v_i)}{(\zeta_g(Q))^2} 
        = \sum_{v_i \in \operatorname{out}(v)} T_g(H, v_i)$$

  - for each edge $v_i$ in $g$ with $i \in \{X,Y,Z\}$:  
    $${\Pr}_{g'}[v_i] = \frac{1}{\lambda_v} \sum_{Q\in H} (\alpha_Q)^2 \frac{\zeta(Q, v_i)}{(\zeta(Q))^2}
        = \frac{T_{g}(H, v_i)}{T_{g}(H, v_X) + T_{g}(H, v_Y) + T_{g}(H, v_Z)}$$ 

- update $g$ for all edges $v_i$: ${\Pr}_g[v_i] \gets (1-\delta)\cdot {\Pr}_g[v_i] + \delta\cdot{\Pr}_{g'}[v_i]$

In [4]:
import copy
import networkx as nx


def T(graph: nx.MultiDiGraph, H: Dict[PauliTerm, d.Decimal], node: Node, pauli_op: PauliOp) -> d.Decimal:
    res_sum = d.Decimal(0)
    for Q, coeff in H.items():
        if coeff == 0:
            continue
        pr = dd.compute_covered_prob(graph, 0, Q)
        pr_vi = dd.compute_covered_prob_through_edge(graph, Q, node, pauli_op)
        #if pr > 0:
        #    print('    {}:{:+.4f} pr={:.4f} pr({}, {})={:.4f}'.format(
        #        Q, 
        #        coeff,
        #        pr,
        #        node,
        #        pauli_op,
        #        pr_vi
        #    ))
        summand = (coeff**2) * pr_vi / (pr**2)
        #print(f'{Q} ==> ({coeff}**2) * {pr_vi} / ({pr}**2) = {summand}')
        res_sum += summand
    #print(f'T(..., {node}, {pauli_op}) = {res_sum}')
    return res_sum


def Txyz(graph: nx.MultiDiGraph, H: Dict[PauliTerm, d.Decimal], node: Node, pauli_ops: Set[PauliOp]) -> Dict[PauliOp, d.Decimal]:
    xyz = {p: d.Decimal(0) for p in pauli_ops}
    for Q, coeff in H.items():
        if coeff == 0:
            continue
        pr = dd.compute_covered_prob(graph, 0, Q)
        part = (coeff**2) / (pr**2)
        
        for i in xyz.keys():
            pr_vi = dd.compute_covered_prob_through_edge(graph, Q, node, i)
            xyz[i] += part * pr_vi        
    return xyz


def optimize_probability_distribution(graph: nx.MultiDiGraph, H: Dict[PauliTerm, d.Decimal], delta: d.Decimal, max_iterations: Optional[int]=None) -> int:
    # new graph with same structure to serve as p'
    new_graph = copy.deepcopy(graph)
    H = {pt: coeff for pt, coeff in H.items() if any(p != 'I' for p in pt)}
    
    converged = False
    i = 0
    while not converged and (max_iterations is None or i < max_iterations):
        if max_iterations is not None:
            print(',', end='')
        i += 1

        converged = True
        # iterate through new graph an set the weights
        for node in new_graph.nodes:
            #print(f'node {node}')
            out_edges = new_graph.out_edges(node, data=True)
            #T_cache = {data['pauli']: T(graph, H, u, data['pauli']) for u, v, data in out_edges}
            T_cache = Txyz(graph, H, node, {data['pauli'] for _, _, data in out_edges})
            T_normalizer = sum(T_cache.values())
            for u, v, data in out_edges:
                data['weight'] = T_cache[data['pauli']] / T_normalizer
            
            weight_out = sum(data['weight'] for u, v, data in new_graph.out_edges(node, data=True))
            assert node == -1 or abs(weight_out - 1) < 0.00001, f'sum == {weight_out} for node {node}'

        # iterate through edges of old graph and adjust the weights 
        for u, v, data in graph.edges(data=True):
            old_weight = data['weight']
            data['weight'] = (1-delta)*data['weight'] + delta*new_graph.edges[u,v,data['pauli']]['weight']            
            if abs(old_weight - data['weight']) >= delta**5:
                converged = False        
    return i
    

# Code for the Benchmarks

In [5]:
import pathlib
import random
import time
import pickle
import datetime
from typing import List
from pathlib import Path

import tabulate
from IPython.core.display import display, HTML
import numpy as np
import qutip
import matplotlib.pyplot as plt
import src.sampling as dd


def read_hamiltonian(filename):
    H = {}
    lineno = 0
    with open(filename, "r") as fp:
        for line in fp:
            if lineno % 2 == 0:
                key = line.rstrip()
            else:
                val = d.Decimal(line.strip())
                #if key in H: print(f'{key}={val} with previous {key}={H[key]}')
                H[key] = val
            lineno += 1
    return H


def convert_hamiltonian_dict_to_matrix(H, nQubits = None, eigenj = None):
    """converting dictionary to matrix, and returning the j-th eigenpairs"""
    I, X, Y, Z = qutip.identity(2), qutip.sigmax(), qutip.sigmay(), qutip.sigmaz()
    paulis = (I, X, Y, Z)
    dPaulis = {"I": 0, "X": 1, "Y": 2, "Z": 3}
    if nQubits is None:
        for k in H.keys():
            nQubits = len(k)
            break
    answer = None
    for k,v in H.items():
        if answer is None:
            answer = float(v) * qutip.tensor( [ paulis[dPaulis[j]] for j in k ] )
        else:
            answer += float(v) * qutip.tensor( [ paulis[dPaulis[j]] for j in k ] )
    eigenpair = answer.eigenstates()
    if eigenj is None:
        return answer, (eigenpair[0][0], eigenpair[1][0])
    else:
        return answer, (eigenpair[0][eigenj], eigenpair[1][eigenj])
       
        
def benchmark_files(files: List[str], prune_below = 0) -> None:
    Path('output').mkdir(exist_ok=True)
    Path('pickle').mkdir(exist_ok=True)
    print(f'###\n### {sum(1 for f in filenames if f)} benchmarks \n###\n')
    summary = {}
    for filename in filenames:
        if filename == '':
            if summary:
                print(tabulate.tabulate([(key, *values) for key, values in summary.items()], 
                    ['Benchmark', '|H|', '|V|', '|E|', '|Eᵥ|', '|Paths|', 'Iter.', 't', 'Var 1S Paths', 'Var 1S Coeff'],
                    tablefmt="simple", floatfmt=('', '', '', '', '', '', '', '.1f', '.4f', '.4f'))
                )
                print('\n')
            continue
        
        start = time.time()
        print(f'{time.strftime("%Y-%m-%d %H:%M:%S")} {filename}')
        print(f'{time.time()-start:.1f}s Reading file into dict...', end='')
        Hdict = read_hamiltonian(filename)
        
        sum_alpha = sum(abs(v) for k, v in Hdict.items())
        sum_alpha_wo_I = sum(abs(v) for k, v in Hdict.items() if any(p != 'I' for p in k))
        print(f' |Hdict|={len(Hdict)}; Σ|α|={sum_alpha:.4f}; Σ|α|-|I|={sum_alpha_wo_I:.4f}')        
        #for k, v in sorted(Hdict.items(), key=lambda e: abs(e[1]), reverse=True):
        #    print(f'{k} => {v}')
        
        print(f'{time.time()-start:.1f}s Creating graph...', end='')
        #graph, ppaths = dd.read_file_to_graph_with_ldf_info(Path(filename))
        graph, ppaths = dd.read_dict_to_graph(Hdict)
        graph.name = pathlib.Path(filename).stem
        dd.normalize_graph(graph, ppaths)
        
        iterations = 0
        # comment the next line to run without optimizations
        iterations = optimize_probability_distribution(graph, Hdict, d.Decimal('0.05'), max_iterations=10)
                
        dd.write_to_dot(graph, 'output/{}.dot'.format(graph.name))
        print(' iterations={}; |V|={}; |E|={}; |Eᵥ|={}; |Paths|={}'.format(
            iterations,
            graph.number_of_nodes(),
            graph.number_of_edges(),
            sum(1 for u, v, data in graph.edges(data=True) if data.get('virtual', False)),
            sum(1 for _ in nx.all_simple_paths(graph, 0, -1)),
        ))
        
        assert dd.check_paths_and_graph_with_wildcard(graph, set(p for p in Hdict.keys() if abs(Hdict[p]) > 0))
        
        #for k, v in sorted(dd.graph_to_dict(graph).items(), key=lambda e: e[1], reverse=True):
        #    print(f'{k} => {v}')
        
        summary[graph.name] = [
            len(Hdict),
            graph.number_of_nodes(),
            graph.number_of_edges(),
            sum(1 for u, v, data in graph.edges(data=True) if data.get('virtual', False)),
            sum(1 for _ in nx.all_simple_paths(graph, 0, -1)),
            iterations,
        ]
         
        #print(f'{time.time()-start:.2f}s done\n')
        #continue
        
        matrix_path = Path(f'pickle/{graph.name}.pickle')
        if matrix_path.exists():
            print(f'{time.time()-start:.1f}s Restoring matrix/eigenstate from {matrix_path}...', end='')
            Hmatrix, eigenpair = pickle.load( matrix_path.open('rb') )
            print(f' exact value={eigenpair[0]}')
        else:
            print(f'{time.time()-start:.1f}s Converting hamiltonian dict...', end='')
            Hmatrix, eigenpair = convert_hamiltonian_dict_to_matrix(Hdict)
            print(f' exact value={eigenpair[0]}; |Hdict|={len(Hdict)}')
            
            print(f'{time.time()-start:.1f}s Dumping matrix/eigenpair to {matrix_path}...', end='')
            pickle.dump( (Hmatrix, eigenpair), matrix_path.open('wb'))
            print(f' done')
        
        print(f'{time.time()-start:.1f}s Running one-path-uniform variance...', end='')
        var_oneshot_paths = one_shot_variance_paths(Hdict, Hmatrix, graph, eigenpair[1], exact_value=d.Decimal(eigenpair[0]))
        print(f' p1={var_oneshot_paths[0]:.7f}; p2={var_oneshot_paths[1]:.7f}; var={var_oneshot_paths[2]:.7f}')
                
        print(f'{time.time()-start:.1f}s Running one-coefficients variance...', end='')
        var_oneshot_coeff = one_shot_variance_coeff(Hdict, Hmatrix, graph, eigenpair[1], exact_value=d.Decimal(eigenpair[0]))
        print(f' p1={var_oneshot_coeff[0]:.7f}; p2={var_oneshot_coeff[1]:.7f}; var={var_oneshot_coeff[2]:.7f}')
        
        print(f'{time.time()-start:.1f}s done\n')
        
        summary[graph.name].extend([
            time.time()-start,
            var_oneshot_paths[2], 
            var_oneshot_coeff[2],
        ])
    
    if summary:
        print(tabulate.tabulate([(key, *values) for key, values in summary.items()], 
            ['Benchmark', '|H|', '|V|', '|E|', '|Eᵥ|', '|Paths|', 'Iter.', 't', 'Var 1S Paths', 'Var 1S Coeff'],
            tablefmt="simple", floatfmt=('', '', '', '', '', '', '', '.1f', '.4f', '.4f'))
        )
    else:
        print('Done but no summary.')


# Benchmarks

Benchmarks under `benchmarks/Hamiltonians` taken from https://github.com/rraymondhp/variances/tree/master/Hamiltonians reformated with
```bash
for file in */*_grouped.txt; do
  dir=$(dirname "$file")
  base=$(basename "$file")
  newname="${dir}_${base//_grouped}"
  < "$file" awk 'BEGIN{FS=","} {printf "%s\n%s\n",$2,$1;}' > "${newname}"
done
```

In [6]:
filenames = [  
    'hamiltonians/H2_STO3g_4qubits_jw.txt', 
    'hamiltonians/H2_STO3g_4qubits_parity.txt', 
    'hamiltonians/H2_STO3g_4qubits_bk.txt',
    
    '', # empty string prints summary
    
    'hamiltonians/H2_6-31G_8qubits_jw.txt', 
    'hamiltonians/H2_6-31G_8qubits_parity.txt',
    'hamiltonians/H2_6-31G_8qubits_bk.txt', 
    
    #'', # empty string prints summary
    
    #'hamiltonians/LiH_STO3g_12qubits_jw.txt', 
    #'hamiltonians/LiH_STO3g_12qubits_parity.txt',
    #'hamiltonians/LiH_STO3g_12qubits_bk.txt', 
    
    #'',
    
    #'hamiltonians/BeH2_STO3g_14qubits_jw.txt', 
    #'hamiltonians/BeH2_STO3g_14qubits_parity.txt',
    #'hamiltonians/BeH2_STO3g_14qubits_bk.txt', 
    
    #'',
    
    #'hamiltonians/H2O_STO3g_14qubits_jw.txt', 
    #'hamiltonians/H2O_STO3g_14qubits_parity.txt',
    #'hamiltonians/H2O_STO3g_14qubits_bk.txt', 
    
    #'',
    
    #'hamiltonians/NH3_STO3g_16qubits_jw.txt', 
    #'hamiltonians/NH3_STO3g_16qubits_parity.txt', 
    #'hamiltonians/NH3_STO3g_16qubits_bk.txt', 
    
    #'',
    
    #'hamiltonians/C2_STO3g_20qubits_jw.txt', 
    #'hamiltonians/C2_STO3g_20qubits_parity.txt', 
    #'hamiltonians/C2_STO3g_20qubits_bk.txt', 
    
    #'',
    
    #'hamiltonians/HCl_STO3g_20qubits_jw.txt', 
    #'hamiltonians/HCl_STO3g_20qubits_parity.txt', 
    #'hamiltonians/HCl_STO3g_20qubits_bk.txt', 
]

benchmark_files(filenames)

###
### 6 benchmarks 
###

2021-05-10 18:42:43 hamiltonians/H2_STO3g_4qubits_jw.txt
0.0s Reading file into dict... |Hdict|=16; Σ|α|=2.7050; Σ|α|-|I|=1.8945
0.0s Creating graph...,,,,,,,,,, iterations=10; |V|=10; |E|=12; |Eᵥ|=0; |Paths|=5
0.3s Restoring matrix/eigenstate from pickle/H2_STO3g_4qubits_jw.pickle... exact value=-1.8572750302023784
0.3s Running one-path-uniform variance... p1=7.6167913; p2=-1.8572750; var=4.1673207
0.3s Running one-coefficients variance... p1=3.8475149; p2=-1.8572750; var=0.3980444
0.3s done

2021-05-10 18:42:44 hamiltonians/H2_STO3g_4qubits_parity.txt
0.0s Reading file into dict... |Hdict|=15; Σ|α|=2.7050; Σ|α|-|I|=1.8945
0.0s Creating graph...,,,,,,,,,, iterations=10; |V|=7; |E|=7; |Eᵥ|=0; |Paths|=2
0.2s Restoring matrix/eigenstate from pickle/H2_STO3g_4qubits_parity.pickle... exact value=-1.8572750302023784
0.2s Running one-path-uniform variance... p1=4.0991471; p2=-1.8572750; var=0.6496766
0.2s Running one-coefficients variance... p1=3.7497288; p2=-1.857